In [1]:
import re
import xml.sax

class ChineseHandler(xml.sax.ContentHandler):
    def __init__(self, out = None):
        self._out = out
        self._current_elem = ''
        self._word = None
    
    def startElement(self, name, attrs):
        self._current_elem = name

    def endElement(self, name):
        self._current_elem = ''
    
    def characters(self, content):
        if self._current_elem == 'title':
            self._word = content
        if self._current_elem == 'text':
            if not re.match(r'[\u4e00-\u9FFF]', self._word):
                return None
            result = re.search(r'\*(近義詞|近义词)：(.+)', content)
            if result:
                synonyms = self._parse_synonyms(result.group(2))
                self._out.write('{}: {}\n'.format(self._word, (', ').join(synonyms)))
                
    def _clean_token(self, t):
        result = re.match(r'\[\[(.+)\]\](.*)', t)
        if result:
            return result.group(1) + result.group(2)
        else:
            return t

    def _parse_synonyms(self, s):
        result = [self._clean_token(t) for t in re.split(r'、|｜|，', s)]
        # print(result)
        return result

In [2]:
hd = ChineseHandler()
assert hd._parse_synonyms('[[生存]]｜[[活]]｜[[生命]]｜[[生計]]') == ['生存', '活', '生命', '生計']
assert hd._parse_synonyms('[[天子]]、[[皇帝]]') == ['天子', '皇帝']
assert hd._parse_synonyms('觀音，觀世音菩薩，觀音菩薩，慈航真人，觀音大士') == ['觀音', '觀世音菩薩', '觀音菩薩', '慈航真人', '觀音大士']
assert hd._parse_synonyms("(單身漢）[[光杆]]｜[[光蛋]]｜[[單身漢]]｜[[光身漢]]｜[[寡佬]]（粵）") == ['(單身漢）[[光杆]]', '光蛋', '單身漢', '光身漢', '寡佬（粵）']
# {{zh-l|無畏}}、{{zh-l|英勇}}、{{zh-l|勇敢}}

In [3]:
with open('zhwiktionary-20200301-pages-articles-multistream.xml') as inp:
    with open('synonyms.txt', 'w') as out:
        handler = ChineseHandler(out)
        xml.sax.parse(inp, handler)

In [4]:
with open('synonyms.txt') as lines:
    words = {}
    for l in lines:
        word, synonyms = l.split(': ')
        words[word] = synonyms.split(', ')
    
    print('Top 10 by word length:')
    print('; '.join(sorted(words.keys(), key=len, reverse=True)[:10]))
    print()
    print('Top 10 by synonyms count:')
    for w, s in sorted(words.items(), key=lambda w: len(w[1]), reverse=True)[:10]:
        print('-', w, len(s))

Top 10 by word length:
百足之虫，至断不僵; 一人之下，万人之上; 擋人財路如殺人父母; 一兼洗褲二兼洗蛤仔; 臨陣磨槍不亮也光; 香港特別行政區; 这山望着那山高; 打斷手骨顛倒勇; 虎落平陽被犬欺; 屋漏偏逢連夜雨

Top 10 by synonyms count:
- 性交 76
- 陰莖 42
- 蝌蚪 19
- 弁言 17
- 妻子 15
- 我 15
- 月經 14
- 揶揄 12
- 犧牲 12
- 伏祈 11


**Про китайску мову:**
- Існує спрощений та традиційний варіант китайської писемності
- в китайскій існує 3 види синонімів:
    - 等义词 | děngyìcí | тотожні слова
    - 同义词 | tóngyìcí | слова близькі за значенням
    - 近义词 | jìnyìcí | майже синонім
- в юнікоді китайска письменість позначається словом Han

**Спостереження:**
- в дампі містилися інші мови: японська, корейська, декілька європейських мов, російська, арабська і т.д.
- використовувалась як спрощена так і традіційна китайска
- у багатьох слів немає синонімів
- деяких слів не існує або вони написані не правильно
- багато синонімів взагалі не підходять словами

Звіряв отримані результати зі словником https://bkrs.info